In [27]:
import numpy as np
import os
import matplotlib.pyplot as plt
import skvideo.io
import cv2
from scipy import ndimage
import skimage.color as cl

In [28]:
#Change current working directory
imagePath = "C:/Users/antonkout/Documents/Publications/paper/synchronized/new"
os.chdir(imagePath)

In [29]:
exec(open("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/helpers.py").read()) 

In [30]:
rgb_mix1_sync = skvideo.io.vread("./rgb_mix1_sync.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
thermal_mix1_sync = skvideo.io.vread("./thermal_mix1_sync.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]

In [31]:
def background_sub_Selective(in_vid, out_vid, par):
    cap = cv2.VideoCapture(in_vid)
    _,frame = cap.read()
    size = (int(cap.get(3)),int(cap.get(4))) 
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(out_vid, fourcc, 25.0, size, isColor=0)
    avg = np.float32(frame)
    
    while(True):
        _,frame = cap.read()
        
        if _ == True:
            cv2.accumulateWeighted(frame,avg,par)
            res = cv2.convertScaleAbs(avg)
            res = cv2.cvtColor(res, cv2.COLOR_RGB2GRAY)
            out.write(res)
        
        else:
            break
        
        if cv2.waitKey(80) & 0xFF == ord('q'):
            break
    
    cv2.destroyAllWindows()
    cap.release()
    out.release()

In [32]:
def background_sub_MOG(in_vid, out_vid, par):
    cap = cv2.VideoCapture(in_vid)
    subtractor = cv2.bgsegm.createBackgroundSubtractorMOG(history=25, nmixtures = 5, backgroundRatio = par, noiseSigma = 0)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,1))
    size = (int(cap.get(3)),int(cap.get(4))) 
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(out_vid, fourcc, 25.0, size, isColor=0)

    while True:
        _, frame = cap.read()

        if _ == True:
            mask = subtractor.apply(frame)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            out.write(mask)
        else:
            break

        if  cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    cap.release()
    out.release()

In [33]:
####Selective Background Subtraction####

In [74]:
#RGB
in_vid = 'rgb_mix1_sync.mp4'
out_vid = "./Selective/rgb_mix1_sel.mp4"
out_vid2 = "./Selective/rgb_mix1_sel2.mp4"
par = 0.000001
par2 = 0.3
background_sub_Selective(in_vid, out_vid, par)
background_sub_Selective(in_vid, out_vid2, par2)
in_vid = 'thermal_mix1_sync.mp4'
out_vid = "./Selective/thermal_mix1_sel.mp4"
out_vid2 = "./Selective/thermal_mix1_sel2.mp4"
background_sub_Selective(in_vid, out_vid, par)
background_sub_Selective(in_vid, out_vid2, par2)

In [ ]:
####MOG Background Subtraction####

In [14]:
#RGB
in_vid = 'rgb_mix1_sync.mp4'
out_vid = "./MOG/rgb_mix1_mog.mp4"
out_vid2 = "./MOG/rgb_mix1_mog2.mp4"
par = 0.001
par2 = 0.2
background_sub_MOG(in_vid, out_vid, par)
background_sub_MOG(in_vid, out_vid2, par2)
in_vid = 'thermal_mix1_sync.mp4'
out_vid = "./MOG/thermal_mix1_mog.mp4"
out_vid2 = "./MOG/thermal_mix1_mog2.mp4"
background_sub_MOG(in_vid, out_vid, par)
background_sub_MOG(in_vid, out_vid2, par2)

In [75]:
rgb_mix1_sel = skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/Selective/rgb_mix1_sel.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
thermal_mix1_sel = skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/Selective/thermal_mix1_sel.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
rgb_mix1_sel2= skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/Selective/rgb_mix1_sel2.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
thermal_mix1_sel2 = skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/Selective/thermal_mix1_sel2.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]

In [76]:
rgb_mix1_sync = skvideo.io.vread("./rgb_mix1_sync.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
thermal_mix1_sync = skvideo.io.vread("./thermal_mix1_sync.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
rgb_mix1_sync = rgb_mix1_sync[:-1,:,:]
thermal_mix1_sync = thermal_mix1_sync[:-1,:,:]

In [77]:
dif = np.zeros(shape=rgb_mix1_sel.shape)
thr = np.zeros(shape=rgb_mix1_sel.shape)
c = np.zeros(shape=rgb_mix1_sel.shape[0])

for k in range(rgb_mix1_sel.shape[0]):    
    dif[k] = rgb_mix1_sync[k]-rgb_mix1_sel[k]
    dif[k][np.where(dif[k]>235)]=0
    c[k],thr[k] = cv2.threshold(dif[k],100,255,cv2.THRESH_BINARY)
del c
skvideo.io.vwrite('./Selective/rgb_diff_sel.mp4', thr)

dif = np.zeros(shape=rgb_mix1_sel.shape)
thr = np.zeros(shape=rgb_mix1_sel.shape)
c = np.zeros(shape=rgb_mix1_sel.shape[0])

for k in range(rgb_mix1_sel.shape[0]):    
    dif[k] = rgb_mix1_sync[k]-rgb_mix1_sel2[k]
    dif[k][np.where(dif[k]>235)]=0
    c[k],thr[k] = cv2.threshold(dif[k],100,255,cv2.THRESH_BINARY)
del c
skvideo.io.vwrite('./Selective/rgb_diff_sel2.mp4', thr)

In [78]:
dif = np.zeros(shape=thermal_mix1_sel.shape)
thr = np.zeros(shape=thermal_mix1_sel.shape)
c = np.zeros(shape=thermal_mix1_sel.shape[0])

for k in range(thermal_mix1_sel.shape[0]):    
    dif[k] = thermal_mix1_sync[k]-thermal_mix1_sel[k]
    dif[k][np.where(dif[k]>235)]=0
    c[k],thr[k] = cv2.threshold(dif[k],100,255,cv2.THRESH_BINARY)
del c
skvideo.io.vwrite('./Selective/thermal_diff_sel.mp4', thr)

dif = np.zeros(shape=thermal_mix1_sel2.shape)
thr = np.zeros(shape=thermal_mix1_sel2.shape)
c = np.zeros(shape=thermal_mix1_sel2.shape[0])

for k in range(thermal_mix1_sel2.shape[0]):    
    dif[k] = thermal_mix1_sync[k]-thermal_mix1_sel2[k]
    dif[k][np.where(dif[k]>235)]=0
    c[k],thr[k] = cv2.threshold(dif[k],100,255,cv2.THRESH_BINARY)
del c
skvideo.io.vwrite('./Selective/thermal_diff_sel2.mp4', thr)

In [39]:
#Create Background and Foreground Scenes

In [172]:
thermal_mix1_sel = skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/Selective/thermal_diff_sel.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
rgb_mix1_sel = skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/Selective/rgb_diff_sel.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
thermal_mix1_sel2 = skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/Selective/thermal_diff_sel2.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
rgb_mix1_sel2 = skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/Selective/rgb_diff_sel2.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]

In [173]:
thermal_mix1_mog = skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/MOG/thermal_mix1_mog.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
rgb_mix1_mog = skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/MOG/rgb_mix1_mog.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
thermal_mix1_mog2 = skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/MOG/thermal_mix1_mog2.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
rgb_mix1_mog2 = skvideo.io.vread("C:/Users/antonkout/Documents/Publications/paper/synchronized/new/MOG/rgb_mix1_mog2.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]

In [174]:
for k in range(thermal_mix1_sel.shape[0]):
    thermal_mix1_sel[k] = np.sign(thermal_mix1_sel[k])    
    rgb_mix1_sel[k] = np.sign(rgb_mix1_sel[k])
    thermal_mix1_sel2[k] = np.sign(thermal_mix1_sel2[k])    
    rgb_mix1_sel2[k] = np.sign(rgb_mix1_sel2[k])  
    thermal_mix1_mog[k] = np.sign(thermal_mix1_mog[k])    
    rgb_mix1_mog[k] = np.sign(rgb_mix1_mog[k])
    thermal_mix1_mog2[k] = np.sign(thermal_mix1_mog2[k])    
    rgb_mix1_mog2[k] = np.sign(rgb_mix1_mog2[k])  
    
background = np.zeros(shape=thermal_mix1_sel.shape)
for k in range(thermal_mix1_sel.shape[0]):
    background[k] = rgb_mix1_sel[k] + rgb_mix1_sel2[k] + thermal_mix1_sel2[k] + rgb_mix1_mog[k] + rgb_mix1_mog2[k] + thermal_mix1_mog[k] + thermal_mix1_mog2[k] 
    background[k] = np.where(background[k]>1, 1, background[k])
    background[k] = np.where(background[k]==1, 255, background[k])

skvideo.io.vwrite("./Selective/background.mp4", background)
    
gas = np.zeros(shape=thermal_mix1_sel.shape)
for k in range(thermal_mix1_sel.shape[0]):
    gas[k] = thermal_mix1_sel[k]
    gas[k] = np.where(gas[k]>1, 1, gas[k])
    gas[k] = np.where(gas[k]==1, 255, gas[k])

skvideo.io.vwrite("./Selective/gas.mp4", gas)
    
gas_detect = np.zeros(shape=thermal_mix1_sel.shape)
for k in range(thermal_mix1_sel.shape[0]):
    gas_detect[k] = gas[k] - background[k]
    gas_detect[k] = np.where(gas_detect[k]==1, 255, gas_detect[k])
    gas_detect[k] = np.where(gas_detect[k]==(-1), 0, gas_detect[k])
skvideo.io.vwrite("./Selective/gas_detect_sel.mp4", gas_detect)

In [140]:
a = 100
b = 255
for k in range(thermal_mix1_sel.shape[0]):
    thermal_mix1_sel[k] = np.sign(cv2.threshold(thermal_mix1_sel[k],a,b,cv2.THRESH_BINARY)[1])    
    rgb_mix1_sel[k] = np.sign(cv2.threshold(rgb_mix1_sel[k],a,b,cv2.THRESH_BINARY)[1])
    thermal_mix1_sel2[k] = np.sign(cv2.threshold(thermal_mix1_sel2[k],a,b,cv2.THRESH_BINARY)[1])    
    rgb_mix1_sel2[k] = np.sign(cv2.threshold(rgb_mix1_sel2[k],a,b,cv2.THRESH_BINARY)[1])  
    thermal_mix1_mog[k] = np.sign(cv2.threshold(thermal_mix1_mog[k],a,b,cv2.THRESH_BINARY)[1])    
    rgb_mix1_mog[k] = np.sign(cv2.threshold(rgb_mix1_mog[k],a,b,cv2.THRESH_BINARY)[1])
    thermal_mix1_mog2[k] = np.sign(cv2.threshold(thermal_mix1_mog2[k],a,b,cv2.THRESH_BINARY)[1])    
    rgb_mix1_mog2[k] = np.sign(cv2.threshold(rgb_mix1_mog2[k],a,b,cv2.THRESH_BINARY)[1])  
    
background = np.zeros(shape=thermal_mix1_sel.shape)
for k in range(thermal_mix1_sel.shape[0]):
    background[k] = rgb_mix1_sel[k] + rgb_mix1_sel2[k] + thermal_mix1_sel2[k] + rgb_mix1_mog[k] + rgb_mix1_mog2[k] + thermal_mix1_mog[k] + thermal_mix1_mog2[k] 
    background[k] = np.where(background[k]>1, 1, background[k])
    background[k] = np.where(background[k]==1, 255, background[k])

skvideo.io.vwrite("./Selective/background.mp4", background)
    
gas = np.zeros(shape=thermal_mix1_sel.shape)
for k in range(thermal_mix1_sel.shape[0]):
    gas[k] = thermal_mix1_sel[k]
    gas[k] = np.where(gas[k]>1, 1, gas[k])
    gas[k] = np.where(gas[k]==1, 255, gas[k])

skvideo.io.vwrite("./Selective/gas.mp4", gas)
    
gas_detect = np.zeros(shape=thermal_mix1_sel.shape)
for k in range(thermal_mix1_sel.shape[0]):
    gas_detect[k] = gas[k] - background[k]
    gas_detect[k] = np.where(gas_detect[k]==1, 255, gas_detect[k])
    gas_detect[k] = np.where(gas_detect[k]==(-1), 0, gas_detect[k])
skvideo.io.vwrite("./Selective/gas_detect_sel.mp4", gas_detect)

In [82]:
#Morphological Operations#

In [175]:
gas_detect = skvideo.io.vread("./Selective/gas_detect_sel.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))

for k in range(gas_detect.shape[0]):
    gas_detect[k] = cv2.morphologyEx(gas_detect[k], cv2.MORPH_OPEN, kernel)
    #gas_detect[k] = cv2.morphologyEx(gas_detect[k], cv2.MORPH_CLOSE, kernel2)
skvideo.io.vwrite("./Selective/gas_detect_morph.mp4", gas_detect)

In [176]:
#Binary Connected Components
gas_detect = skvideo.io.vread("./Selective/gas_detect_morph.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
gas_detect = bcc_fun(gas_detect,"./Selective/gas_detect_bcc.mp4",50)

In [177]:
#Apply mask to thermal and rgb videos

In [178]:
rgb_mix1_sync = skvideo.io.vread("./rgb_mix1_sync.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
thermal_mix1_sync = skvideo.io.vread("./thermal_mix1_sync.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
gas_detect = skvideo.io.vread("./Selective/gas_detect_bcc.mp4",outputdict={"-pix_fmt": "gray"})[:, :, :, 0]

In [179]:
video = thermal_mix1_sync.copy()
video = np.asarray(video, dtype=np.uint8)
for k in range (gas_detect.shape[0]):
    for i in range (gas_detect.shape[1]):
        for j in range (gas_detect.shape[2]):
            if (gas_detect[k,i,j]!=0):
                gas_detect[k,i,j] = thermal_mix1_sync[k,i,j]

pseudo_color = np.zeros(shape=(gas_detect.shape[0],gas_detect.shape[1],gas_detect.shape[2],3))
for k in range (gas_detect.shape[0]):
    pseudo_color[k] = cv2.applyColorMap(gas_detect[k], cv2.COLORMAP_JET)

In [180]:
gas_detect_color = np.zeros(shape=(gas_detect.shape[0],gas_detect.shape[1],gas_detect.shape[2],3))
for k in range(gas_detect.shape[0]):
    gas_detect_color[k] = cl.gray2rgb(thermal_mix1_sync[k])
gas_detect_color = np.asarray(gas_detect_color, dtype=np.uint8)
for k in range (pseudo_color.shape[0]):
    for i in range (pseudo_color.shape[1]):
        for j in range (pseudo_color.shape[2]):
#            for h in range (pseudo_color.shape[3]):
            if(pseudo_color[k,i,j,0]!=128):
                gas_detect_color[k,i,j,0] = pseudo_color[k,i,j,0]

In [181]:
skvideo.io.vwrite("./Selective/gas_detect_color.mp4", gas_detect_color)